### **4.12 - How to deal with failures in your DAGs**

En un mundo perfecto nunca experimentaremos ningún fallo. Todo funcionará como se espera y usted será el ingeniero más feliz del trabajo. Lo siento, pero no es así en absoluto. Como ingeniero, sabemos lo importante que es hacer frente a los fallos, ya que pueden producirse en cualquier momento y romper todo el data pipeline. Dado que Airflow es el orquestador de sus data pipelines, es crítico ser advertido cuando una tarea no tuvo éxito o si uno de sus DAGs está siendo más lento en terminar que nosotros y así sucesivamente. Lo bueno es que Airflow nos ofrece muchas formas de monitorizar nuestros DAGs y asegurarnos de que todo funciona como debería. **`Hay dos niveles de detección de fallos, uno para los DAGs y otro para las tareas`**.

#### **`DAG failure detections`**

<center><img src="https://i.postimg.cc/kg4s5MDn/a712.png"></center>

___

#### **`dagrun_timeout`**
#### **`max_active_runs`**

Empecemos por la detección de fallos en los DAGs (DAG failure detections). Cuando instancias un DAG, puedes especificar todos estos parámetros para comprobar la ejecución de tus DAGRuns. El parámetro más básico es '**`dagrun_timeout`**' que especifica cuánto tiempo debe estar un DagRun antes de que se agote el timeout, para que se puedan crear nuevos DagRuns. **`Ten en cuenta que este timeout sólo es efectivo para los scheduled DagRuns, por lo que no funcionará si disparas manualmente su DAG, y sólo una vez que el número de DagRuns activos sea igual al parámetro 'max_active_runs'`**.

<center><img src="https://i.postimg.cc/j5zQ1L3h/a713.png"></center>

Entonces, ¿qué es max_active_runs?

Bueno, como veremos más adelante en el curso sobre la distribución de Airflow, **`'max_active_runs'`** **es una propiedad de configuración que permite fijar el número máximo de ejecuciones activas por DAG (DAG runs per DAG). Por ejemplo, digamos que esta propiedad está configurada a 16, que es el caso por defecto**, y usted activa un DAG del pasado. Airflow ejecutará hasta 16 DagRuns en paralelo para este DAG con el fin de lograr el backfilling proceso más rápido.

Dicho esto, realmente depende de ti elegir el valor correcto para el 'dagrun_timeout' según el tiempo que esperas que tome tu DAG. Después de haber disparado tu DAG varias veces, deberías ser capaz de tener el tiempo medio para conseguir que termine y establecer ese valor para el parámetro.

___

#### **`sla_miss_callback`**

El siguiente argumento que tenemos es 'sla_miss_callback'. No voy a explicar esta noción ahora ya que hay un video dedicado sobre el uso de SLA en sus DAGs más adelante en el curso, pero ten en cuenta que sla_miss_callback permite llamar a una función cuando se informa de SLA timeouts.

___

#### **`on_failure_callback`**
#### **`on_success_callback`**

Luego, tenemos 'on_failure_callback' así como 'on_success_callback'. Como se puede adivinar por sus nombres, el primero se utiliza para llamar a una función cuando el DagRun de este DAG falla mientras que la segunda se utiliza para llamar a una función cuando el DagRun tiene éxito. Ambas funciones toman un diccionario de contexto como parámetro donde se puede encontrar información sobre el DagRun así como de las task instances.

___

#### **`Task failure detections`**

- **`email_on_failure`**
- **`email_on_retry`**
- **`retries`**
- **`retry_delay`**
- **`retry_exponential_backoff`**
- **`on_failure_callback`**
- **`on_success_callback`**
- **`on_retry_callback`**

Muy bien, hemos visto las formas de ser alertado a nivel de DAG, veamos qué podemos hacer a nivel de tarea.

<center><img src="https://i.postimg.cc/gjmqbmfb/a714.png"></center>

En primer lugar, tenemos todos los argumentos relacionados con la alerta por correo electrónico. De hecho, si establecemos un correo electrónico y fijamos los parámetros '**`email_on_failure`**' así como '**`email_on_retry`**' a true, recibiremos un correo electrónico tan pronto como una tarea se reintente o falle. Al igual que hicimos anteriormente para el forex data pipeline, si quiere que Airflow envíe correos electrónicos en caso de reintentos y fallos, tiene que configurar el servidor SMTP en el archivo de configuración arflow.cfg. Observe que puede establecer más de un correo electrónico donde enviar las alertas.

A continuación, tenemos todos los parámetros relacionados con el reintento de una tarea. En el mismo orden que se muestra aquí, '**`retries`**' indica el número de reintentos que deben realizarse antes de marcar la tarea como fallida, '**`retry_delay`**' especifica el retraso con un objeto timedelta entre reintentos, '**`retry_exponential_backoff`**' cuando se establece en true, permite esperas progresivas más largas entre reintentos. Y finalmente, '**`max_retry_delay`**' define el intervalo máximo de retardo entre reintentos utilizando un objeto timedelta. Puedes ver este parámetro como un timeout (tiempo de espera) para los reintentos. Si por alguna razón, una tarea tarda más tiempo que 'max_retry_delay' en volver a activarse, el reintento se salta.


Entonces, tenemos **`execution_timeout`** correspondiente al tiempo máximo de ejecución permitido de una instancia de tarea dada expresado con un objeto timedelta. Si una tarea tarda más de execution_timeout en terminar, se marca como fallida.

Los últimos parámetros son los **`callbacks`**. Al igual que para el DAG, podemos especificar una función para llamar cuando una tarea es fallida, exitosa o reintentada.